<a href="https://colab.research.google.com/github/JacobyNevada/ds_belhard/blob/main/hw4/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from google.colab import drive
import os
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

def data_loading(file_link):
  drive.mount('/content/drive')
  file_path = f"/content/drive/{file_link}"

  # Загрузка CSV файла
  dataset = pd.read_csv(file_path)
  return dataset


#################################################################3


# https://www.kaggle.com/datasets/henrysue/online-shoppers-intention
file_link = 'MyDrive/Colab Notebooks/hw4/online_shoppers_intention.csv'
dataset = data_loading(file_link)

print(dataset)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0               